In [ ]:
import os
import pandas as pd
import requests
from pipeline.configs.config import DATA_PATH_TEST, PIPELINE_NAME, LOCAL_RUNNER_PATH, PIPELINE_ROOT_PATH

# Create Pipeline

In [ ]:
!tfx pipeline create --engine=local --pipeline_path={LOCAL_RUNNER_PATH}

In [ ]:
!tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

# Predict Values

In [ ]:
SERVING_MODEL_DIR = os.path.join(PIPELINE_ROOT_PATH, 'serving_model')

In [ ]:
!wget 'http://storage.googleapis.com/tensorflow-serving-apt/pool/tensorflow-model-server-universal-2.8.0/t/tensorflow-model-server-universal/tensorflow-model-server-universal_2.8.0_all.deb'
!sudo dpkg -i --force-overwrite tensorflow-model-server-universal_2.8.0_all.deb

In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=test_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

In [ ]:
data = pd.read_csv(DATA_PATH_TEST)

json_data = data.to_json(orient='records')

url = 'http://localhost:8501/v1/models/{PIPELINE_NAME}:predict'
headers = {'Content-Type': 'application/json'}
response = requests.post(url, data=json_data, headers=headers)

if response.status_code == 200:
    predictions = response.json()
    print(predictions)
else:
    print("Error:", response.status_code)